In [7]:
import json
import requests

In [8]:
env = json.loads(open("env.txt", "r").read())
token_url = 'https://oauth.battle.net/token'

data = {'grant_type': 'client_credentials'}

response = requests.post(token_url, data=data, auth=(env["client_identity"], env["client_secret"]))
# conversion_file = open("conversions.txt", "w")
access_token = response.json().get('access_token')


In [9]:
# Gets meta data
api_url = f"https://us.api.blizzard.com/hearthstone/metadata/?locale=en_US"
headers = {'Authorization': f'Bearer {access_token}'}
with open("all_meta_data.json", "w") as outfile:
	response = requests.get(api_url, headers=headers)
	response_map = {}
	if response.status_code == 200:
		response_json = response.json()
		outfile.write(json.dumps(response_json, indent = 4))
	else:
		print(f"Error: {response.status_code} - {response.text}")


In [10]:
def get_set_data(access_token, set, id):
	count = 1
	cards_in_set = {}
	cards_in_set["id"] = id
	while(True):
		api_url = f"https://us.api.blizzard.com/hearthstone/cards?locale=en_US&set={set}&page={count}"
		count += 1
		headers = {'Authorization': f'Bearer {access_token}'}

		response = requests.get(api_url, headers=headers)
		
		if response.status_code == 200:
			response_json = response.json()
			if(len(response_json["cards"]) == 0):
				break
			for card_data in response_json["cards"]:
				cards_in_set[card_data["id"]] = card_data
		else:
			print(f"Error: {response.status_code} - {response.text}")
			return cards_in_set
	return cards_in_set

In [11]:
data = json.loads(open("all_meta_data.json", "r").read())
for set_descriptor in data["sets"]:
	with open(f"SetData/{set_descriptor['name']}.json", "w") as outfile:
		json_data = get_set_data(access_token,set_descriptor["slug"], set_descriptor["id"])
		outfile.write(json.dumps(json_data, indent=4))